<div class='alert alert-info'>
    <h1 align="center">Course Scheduler</h1>
    <h3 align="center">Mohammad Rahdar</h3>    
</div>

Welcome to the **Course Scheduler App**! 🎓  

<u><a href="https://github.com/mo-rahdar/Course-Scheduler/" target="_blank">GitHub Link</a></u> 

- Upload your **Courses_info.xlsx** file using the button below.  
- Click **Run Scheduler** to generate an optimized course schedule.  
- The results will include:  
  - A saved file `Schedule_results.xlsx` with the schedule.  
  - Plots showing course assignments, conflicts, and available meeting times.  

👉 Once finished, you can download and optionally modify the schedule.  
If you make changes, use the **Schedule Checker App** (see Binder link in README) to validate your edits.


In [ ]:
import traceback
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import Utils
import base64

# Widgets
upload_input = widgets.FileUpload(accept='.xlsx', multiple=False, description='Upload Input Excel')
run_btn = widgets.Button(description='Run Scheduler', button_style='success')

upload_result = widgets.FileUpload(accept='.xlsx', multiple=False, description='Upload Results Excel')
check_btn = widgets.Button(description='Check Schedule', button_style='info')

out = widgets.Output()

# Placeholders for download buttons
download_sched_btn = widgets.HTML("")
download_check_btn = widgets.HTML("")

# Display both workflows
display(widgets.HTML("<h3>1️⃣ Generate a Schedule</h3>"))
display(upload_input, run_btn, download_sched_btn)

display(widgets.HTML("<h3>2️⃣ Check a Modified Schedule</h3>"))
display(upload_result, check_btn, download_check_btn)

display(out)

# Utility to make a download link
def make_download_link(file_path, link_text, filename=None):
    try:
        with open(file_path, "rb") as f:
            data = f.read()
        b64 = base64.b64encode(data).decode()
        filename = filename or file_path.split("/")[-1]
        payload = f'<a download="{filename}" href="data:application/octet-stream;base64,{b64}" target="_blank">{link_text}</a>'
        return payload
    except Exception as e:
        return f"<p style='color:red;'>⚠️ Could not create download link: {e}</p>"

# Callbacks
def on_run_clicked(b):
    with out:
        clear_output()
        if not upload_input.value:
            print("Please upload an Excel (.xlsx) file first.")
            return
        try:
            fname = Utils.save_uploaded_file(upload_input, mode='scheduler')
            print("Saved uploaded file as", fname)

            # Run pipeline
            result_file = Utils.run_pipeline(fname)

            # Show download link
            # download_sched_btn.value = make_download_link(result_file, "⬇️ Download Schedule Result", "Schedule_Result.xlsx")
            display(Utils.make_download_html("Schedule_results.xlsx", "⬇️ Download results (Excel)"))

        except Exception as e:
            print("❌ Error during run:")
            traceback.print_exc()

def on_check_clicked(b):
    with out:
        clear_output()
        if not upload_result.value:
            print("Please upload a Schedule_results.xlsx file first.")
            return
        try:
            fname = Utils.save_uploaded_file(upload_result, mode='check')
            print("Saved uploaded results as", fname)

            # Run check pipeline
            result_file = Utils.run_check_pipeline(fname)

            # Show download link
            download_check_btn.value = make_download_link(result_file, "⬇️ Download Check Report", "Check_Result.xlsx")

        except Exception as e:
            print("❌ Error during check:")
            traceback.print_exc()

# Connect buttons
run_btn.on_click(on_run_clicked)
check_btn.on_click(on_check_clicked)
